---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [239]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here



# read training data
train_df = pd.read_csv('train.csv', encoding='ISO-8859-1')

# analyze features
# train_df['compliance'].unique()
# train_df.describe(include='all')
# train_df.corr()

train_df = train_df[~train_df['compliance'].isna()]
train_df.set_index('ticket_id', inplace=True)

# read location lookups
addresses_df = pd.read_csv('addresses.csv')
latlons_df = pd.read_csv('latlons.csv')
loc_info_df = addresses_df.merge(latlons_df, on='address').drop('address', axis=1)
loc_info_df.set_index('ticket_id', inplace=True)


test_df = pd.read_csv('test.csv')
test_df2 = test_df.merge(loc_info_df, on='ticket_id')
test_df2.set_index('ticket_id', inplace=True)
#test_df2


In [246]:
import time
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

def fill_nan_in_known(df):
    df['violation_zip_code'].fillna(0, inplace=True)
    df['mailing_address_str_number'].fillna(0, inplace=True)
    df['lat'].fillna(train_df2['lat'].mean(), inplace=True)
    df['lon'].fillna(train_df2['lon'].mean(), inplace=True)



# drop additional columns except compliance
drop_cols = train_df.columns.difference(test_df.columns).drop('compliance')
train_df2 = train_df.drop(drop_cols, axis=1)
#train_df2 = train_df2.merge(loc_info_df, on='ticket_id')
train_df2 = train_df2.join(loc_info_df)

""" 
# drop object columns - errors when using encoder later
typ = train_df2.dtypes
str_cols = typ[typ=='object'].index.tolist()
train_df2 = train_df.drop(str_cols, axis=1)
"""

# Fill nan values before using Encoder

# train_df2[[col for col in train_df2 if train_df2.isna().any()[col] ]].dtypes
fill_nan_in_known(train_df2)
# test_df2[[col for col in test_df2 if test_df2.isna().any()[col] ]].dtypes
fill_nan_in_known(test_df2)

# create X and y for training set
y = train_df2['compliance']
train_df3 = train_df2.drop('compliance', axis=1)

""" 
# columns to be scaled
scale_cols = ['violation_street_number', 'fine_amount', 'admin_fee', 'state_fee','late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount']

col_xfm = ColumnTransformer(remainder='passthrough', 
                            transformers=[('minmax', MinMaxScaler(), scale_cols)])
xfm = col_xfm.fit_transform(train_df3)
"""


# Encode object columns
typ = train_df3.dtypes
str_cols = typ[typ=='object'].index.tolist()

encoder = LabelEncoder()
train_df4 = train_df3.copy()
for c in str_cols:
    #print(f'Col={c}')
    if 'zip' in c:
        #print('replacing values in zip')
        train_df4[c] = train_df4[c].str.slice(0, 5)
        train_df4[c] = train_df4[c].str.replace('[^\d]', '', regex=True)
        train_df4[c].fillna(0, inplace=True)
        #X_train2[c] = X_train2[X_train2[c].astype('str').str.contains('[^\d]', regex=True)][c].str.replace('[^\d]', '', regex=True)
        train_df4[c] = train_df4[c].astype('str')
    elif 'date' in c:
        train_df4[c] = pd.to_datetime(train_df4[c])  
        train_df4[c].fillna(train_df4[c].mean(), inplace=True)
    else:
        train_df4[c] = pd.to_numeric(train_df4[c], errors='ignore', downcast='float')
    #if train_df4[c].isnull().all():
    #    print(f"drop={c}")
    #    train_df4.drop(c, inplace=True)
    #else:
        train_df4[c].fillna('000', inplace=True)
    train_df4[c] = encoder.fit_transform(train_df4[c])
        

# create a validation set to check model performance
X_train, X_val, y_train, y_val = train_test_split(train_df4, y, random_state=0)


In [247]:
#pd.to_numeric(X_train2['zip_code'], errors='ignore', downcast='float')
# replace non-numeric zip with nan
#X_train2[X_train2['zip_code'].astype('str').str.contains('[^\d]', flags=re.IGNORECASE, regex=True)]['zip_code'] #.str.replace('[^\d]', '', regex=True)
# replace hyphen in numeric zip
#X_train2[X_train2['zip_code'].str.contains('-', regex=True)==True]['zip_code']  #.str.replace('-.*', '')
#print(X_train2['zip_code'].unique().tolist())
#pd.to_numeric(X_train2['zip_code'] , errors='ignore', downcast='integer')
#X_train2['zip_code'].unique().astype(int)
#pd.to_numeric(X_train2['zip_code'], errors='ignore', downcast='integer').dtype
#X_train2[X_train2['zip_code'].isna()]

#X_train2['violation_zip_code'].fillna(0, inplace=True)
#X_train2['mailing_address_str_number'].fillna(0, inplace=True)
#X_train2[X_train2.isnull().any(axis=1)]

#train_df4[[col for col in train_df4 if train_df4.isna().any()[col] ]].dtypes
#test_df2[[col for col in test_df2 if test_df2.isna().any()[col] ]].dtypes
#train_df4[train_df2.isnull().any(axis=1)][[col for col in train_df4 if train_df4.isna().any()[col] ]]

# violator_name	mailing_address_str_name	state	zip_code	non_us_str_code	hearing_date	grafitti_status
#train_df4[train_df4['mailing_address_str_name'].isnull()]

train_df4.dtypes

agency_name                     int64
inspector_name                  int64
violator_name                   int64
violation_street_number       float64
violation_street_name           int64
violation_zip_code            float64
mailing_address_str_number    float64
mailing_address_str_name        int64
city                            int64
state                           int64
zip_code                        int64
non_us_str_code                 int64
country                         int64
ticket_issued_date              int64
hearing_date                    int64
violation_code                  int64
violation_description           int64
disposition                     int64
fine_amount                   float64
admin_fee                     float64
state_fee                     float64
late_fee                      float64
discount_amount               float64
clean_up_cost                 float64
judgment_amount               float64
grafitti_status                 int64
lat         

In [244]:

# scale features in training set
scaler = MinMaxScaler()
# X_train_scaled = X_train2
# X_train_scaled[scale_cols] = scaler.fit_transform(X_train2[scale_cols])
# X_val_scaled = X_val2
# X_val_scaled[scale_cols] = scaler.transform(X_val2[scale_cols])
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# try different classifiers
classifiers = [LogisticRegression(), GaussianNB(),
               RandomForestClassifier(n_estimators=100, max_features=10, max_depth=7), 
               MLPClassifier(hidden_layer_sizes=[100, 100], random_state=0,
                             activation='relu', alpha=1.0, solver='adam')]
classifiers


X_train_scaled

ValueError: could not convert string to float: 

In [217]:

# check scores of each classifier
for clf in classifiers:
   start_time = time.time()
   fitted = clf.fit(X_train_scaled, y_train)
   y_predict = fitted.predict(X_val_scaled)
   auc = roc_auc_score(y_val, y_predict)
   end_time = time.time()
   print(f'Time taken = {end_time - start_time}, AUC = {auc} for {clf}')
   

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [3]:

# get test data
X_test = test_df

# scale features in test set
#X_test_scaled = scaler.transform(X_test)

# predict test output


# get prediction probablilities



NameError: name 'scaler' is not defined